In [ ]:
# Importation des librairies
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns
import xgboost           as xgb
import torch.nn          as nn
import pickle
import torch
import re
import nltk

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from joblib import dump, load

In [ ]:
from nltk.corpus import stopwords # Les stopwords sont des mots qui n'apportent pas de sens à la phrase
from nltk.stem   import PorterStemmer # Le stemmer permet de réduire les mots à leur racine
from sklearn.feature_extraction.text import CountVectorizer # Permet de transformer le texte en vecteur (bag of words)
from sklearn.metrics import confusion_matrix

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Lecture des données

In [ ]:
# On lis les données préprocessées d'entrainement
df_train = pd.read_csv('../data/train_preprocessed.csv')

In [ ]:
# On lis les données préprocessées de test
df_test = pd.read_csv('../data/test_preprocessed.csv')

In [ ]:
# On convertie les labels en entier pour les données d'entrainement
df_train['label'] = df_train['label'].astype('category').cat.codes

In [ ]:
# On convertie les labels en entier pour les données de test
df_test['label'] = df_test['label'].astype('category').cat.codes

In [ ]:
# On compte le nombre de chaque label
df_train['label'].value_counts()

In [ ]:
# On garde la même proportion de données par label
# df_train = df_train.groupby('label').apply(lambda x: x.sample(n=27253)).reset_index(drop=True)

In [ ]:
df_train

In [ ]:
df_test

# Création des données d'entrainement

In [ ]:
X_train = df_train['content']
y_train = df_train['label']

In [ ]:
demo = df_test['content']

In [ ]:
X_test = df_test['content']
y_test = df_test['label']

# Création du modèle

In [ ]:
# On créer un pipeline qui permet de transformer le texte en vecteur
pipeline_preprocessing = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', xgb.XGBClassifier())
])

In [ ]:
# On entraine le modèle
pipeline_preprocessing.fit(X_train, y_train)

In [ ]:
# On prédit les labels des données de test
y_pred = pipeline_preprocessing.predict(X_train)

In [ ]:
# On affiche le nombre de bonnes prédictions
print(np.sum(y_pred == y_train) / len(y_train))

In [ ]:
# On affiche le nombre de classification 0
print(np.sum(y_pred == 0))

In [ ]:
# On recupère les URLs des données de test ou y_pred = 0
df_train[y_pred == 0]['url'].to_csv('../data/urls.csv', index=False)

In [ ]:
# On affiche le nombre de classification 1
print(np.sum(y_pred == 0))

In [ ]:
# On sauvegarde le modèle
dump(pipeline_preprocessing, '../models/model.joblib')

#